<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/5_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### function

* Grade B/C

In [ ]:
def check_grade_B(df):
    chart_counts = df['CHART'].value_counts()
    df['Grade_B'] = df['CHART'].apply(lambda x: '1' if chart_counts[x] >= 3 else '0')
    return df

def check_grade_C(df):
    df['Grade_C'] = df['monthly_fail_flag'].apply(lambda x: '1' if x == 1 else '0')
    return df

* Grade A

In [ ]:
import pandas as pd
import re

def parse_rule_result(s):
    """
    從 rule_result 字串中提取前兩個中括號中的數值。
    空值或缺失值回傳 0。
    """
    matches = re.findall(r'\[([-\d.]+)?\]', s)
    # 若為 ''或'-999' 則給0，等於無條件通過mean_shift判定
    val1 = float(matches[0]) if matches[0] not in ['', '-999'] else 0
    val2 = float(matches[1]) if matches[1] not in ['', '-999'] else 0
    return val1, val2

def check_grade_A(df):
    # Step 1: 定義分類條件
    is_saw = df['PROGRP'].str.startswith('[SF') | df['PROGRP'].str.startswith('[Tf')
    is_hbtb = df['PROGRP'].isin(['[H01U-PB]', '[H01U-KB]', '[H02U-F2]'])
    is_others = ~(is_saw | is_hbtb)

    # Step 2: 解析 rule_result 數值
    result_vals = df['rule_result'].apply(parse_rule_result)
    df['Mean_shift'] = result_vals.apply(lambda x: x[0])
    df['Sigma_ratio'] = result_vals.apply(lambda x: x[1])

    # Step 3: 建立 Grade_A 欄位
    df['Grade_A'] = '0'

    # SAW 條件：一律為 1
    df.loc[is_saw, 'Grade_A'] = '1'

    # HBTB 條件
    df.loc[is_hbtb & (
        (df.groupby('PROGRP')['param'].transform('count') >= 2) |  # PROGRP 下有param重複（出現2次以上）
        (df['Mean_shift'].abs() > 2) |
        (df['Sigma_ratio'].abs() > 2)
    ), 'Grade_A'] = '1'

    # Others 條件
    df.loc[is_others & (
        (df.groupby(['PROGRP', 'Fab'])['param'].transform('count') >= 2) |  # PROGRP+Fab 下有param重複（出現2次以上）
        (df['Mean_shift'].abs() > 2) |
        (df['Sigma_ratio'].abs() > 2)
    ), 'Grade_A'] = '1'

    df = df.drop(columns = ['Mean_shift', 'Sigma_ratio'])
    return df


### 判定

In [ ]:
from dateutil.relativedelta import relativedelta
import pandas as pd

review_month = '2025M04'
base_on, stat = 'WAFER', 'MEAN'
def add_priority(review_month, pre_record_df, record_df):
  start_month = (pd.to_datetime(review_month.replace('M', ''), format='%YM%m') - relativedelta(months=5)).strftime('%YM%m')
  # 合併前5個月 + 當月資料，並篩選 fail_flag = 1
  combined_df = pd.concat([pre_record_df, record_df], ignore_index=True)
  filtered_df = combined_df[(combined_df['Month'] >= start_month) & (combined_df['fail_flag'] == '1')].reset_index(drop=True)

  # 進行 grade 計算
  filtered_df = check_grade_A(filtered_df)  # 新增 Grade_A 欄位
  filtered_df = check_grade_B(filtered_df)  # 新增 Grade_B 欄位

  # 取出本月份資料, 並做最終 Grade 判斷
  current_df = filtered_df[filtered_df['Month'] == review_month].copy()
  current_df['GRADE'] = current_df.apply(
      lambda row: 'A' if row['Grade_A'] == '1' else ('B' if row['Grade_B'] == '1' else 'C'),
      axis=1
  )

  # 合併回原本 record_df，fail_flag=0 則 Priority = ''
  record_df = record_df.merge(current_df[['CHART', 'GRADE']], on='CHART', how='left')
  record_df['GRADE'] = record_df['GRADE'].fillna('')
  # 移除原 Priority 欄並改名
  record_df = record_df.drop(columns='Priority').rename(columns={'GRADE': 'Priority'})
  return record_df


In [ ]:
record_lst = []
for group, temp_record_df in record_df.groupby(['BASE_ON','STAT']):
  temp_pre_record_df = pre_record_df[(pre_record_df['BASE_ON']==group[0])&(pre_record_df['STAT']==group[1])]
  temp_record_df_new = add_priority(review_month, temp_pre_record_df, temp_record_df)
  record_lst.append(temp_record_df_new)
all_record_df = pd.concat(record_lst, axis=0, ignore_index=True)